In [1]:
from pycaret.utils import version
version()

2.0


In [3]:
import pandas as pd
data = pd.read_csv('./data/cs-training.csv')

In [15]:
from pycaret.classification import *
clf1 = setup(data=data, target='SeriousDlqin2yrs', numeric_features=['NumberOfTime30-59DaysPastDueNotWorse',
                                                                     'NumberOfTimes90DaysLate',
                                                                     'NumberOfTime60-89DaysPastDueNotWorse',
                                                                     'NumberOfDependents'])

Setup Succesfully Completed!


,Description,Value
0,session_id,2689
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(150000, 12)"
4,Missing Values,True
5,Numeric Features,11
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [14]:
# return best model
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.9374,0.8642,0.1872,0.6029,0.2856,0.2622,0.3123,0.5467
1,Gradient Boosting Classifier,0.9371,0.8634,0.1932,0.5911,0.2912,0.2670,0.3136,8.6303
2,CatBoost Classifier,0.9359,0.8632,0.1898,0.5603,0.2834,0.2583,0.3006,7.0050
3,Ada Boost Classifier,0.9357,0.8571,0.2068,0.5499,0.3004,0.2740,0.3105,2.0539
4,Extreme Gradient Boosting,0.9354,0.8571,0.1941,0.5480,0.2864,0.2606,0.2998,5.1748
5,Ridge Classifier,0.9333,0.0000,0.0178,0.5410,0.0344,0.0303,0.0881,0.0194
6,Logistic Regression,0.9332,0.6709,0.0134,0.5253,0.0260,0.0228,0.0744,0.4328
7,Linear Discriminant Analysis,0.9331,0.6922,0.0969,0.4976,0.1621,0.1434,0.1979,0.0679
8,Extra Trees Classifier,0.9329,0.8248,0.1857,0.4957,0.2701,0.2425,0.2751,0.8691
9,Random Forest Classifier,0.9327,0.7801,0.1654,0.4899,0.2472,0.2209,0.2573,0.3117


In [17]:
# tune modal
tuned_clf = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9377,0.8629,0.1897,0.6073,0.2891,0.2658,0.3161
1,0.9362,0.8633,0.1823,0.5714,0.2765,0.2523,0.2983
2,0.9350,0.8564,0.2051,0.5353,0.2966,0.2695,0.3041
3,0.9361,0.8665,0.1866,0.5671,0.2808,0.2562,0.3004
4,0.9363,0.8672,0.2179,0.5604,0.3138,0.2872,0.3229
5,0.9370,0.8682,0.2023,0.5820,0.3002,0.2752,0.3181
6,0.9347,0.8585,0.1766,0.5345,0.2655,0.2403,0.2814
7,0.9380,0.8602,0.1852,0.6220,0.2854,0.2628,0.3167
8,0.9354,0.8594,0.1866,0.5504,0.2787,0.2534,0.2948
9,0.9402,0.8679,0.2325,0.6443,0.3417,0.3176,0.3635


In [18]:
ensembled_clf = ensemble_model(tuned_clf, method = 'Boosting', n_estimators = 100)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9332,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.9331,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
9,0.9332,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000


In [20]:
pred_holdout = predict_model(tuned_clf)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9363,0.8593,0.1915,0.5703,0.2867,0.2619,0.3054


In [21]:
test_data = pd.read_csv('data/cs-test.csv')
predictions = predict_model(pred_holdout, data=test_data)

AttributeError: 'DataFrame' object has no attribute 'predict'

In [23]:
final_clf = finalize_model(tuned_clf)
final_clf

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.2, max_depth=70,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.4,
               n_estimators=200, n_jobs=-1, num_leaves=50, objective=None,
               random_state=2689, reg_alpha=0.6, reg_lambda=0.7, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# initializing setup
from pycaret.classification import *
clf2 = setup(data, target='SeriousDlqin2yrs', numeric_features=['NumberOfTime30-59DaysPastDueNotWorse',
                                                                     'NumberOfTimes90DaysLate',
                                                                     'NumberOfTime60-89DaysPastDueNotWorse',
                                                                     'NumberOfDependents'])

# compare all baseline models and select top 5
top5 = compare_models(n_select = 5)

# tune top 5 base models
tuned_top5 = [tune_model(i) for i in top5]

# ensemble top 5 tuned models
bagged_top5 = [ensemble_model(i) for i in tuned_top5]

# blend top 5 base models
blender = blend_models(estimator_list = top5)

# select best model
best = automl(optimize = 'Recall')

Setup Succesfully Completed!


,Description,Value
0,session_id,779
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(150000, 12)"
4,Missing Values,True
5,Numeric Features,11
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


IntProgress(value=0, description='Processing: ', max=180)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:47:48
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


In [ ]:
!mlflow ui